In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Comparing Two Samples

In [ ]:
births = Table.read_table('https://raw.githubusercontent.com/data-8/materials-sp20-share/master/lec%20notebooks/baby.csv?token=ABGR23JXYMATADGVFXPMQLS7TK2K6')

In [ ]:
births

In [ ]:
smoking_and_birthweight = births.select('Maternal Smoker', 'Birth Weight')

Create a count of how many babies fall into each category for `Maternal Smoker`

In [ ]:
...

Create a histogram of each group of babies. Hint: the `.hist()` function as an optional keyword argument of `group`.

In [ ]:
...

# Test Statistic
Overall Question: Could the difference in the samples be due to chance alone? 

Null: If we had the entire population of new borns,  the birthweights for smokers and non-smokers would be the same.

Alternative: In the population, the mothers who did not smoke had babies that were heavier, on average, than the babies from mothers who did smoke.


**Question 1**: What is a good test statistic?

What values of our statistic are in favor of the alternative: positive or negative?

In [ ]:
...

## Write a function that will compute the observed value of the test statistic

In [ ]:
def difference_of_means(table, label, group_label):
    """Takes: name of table, column label of numerical variable,
    column label of group-label variable
    Returns: Difference of means of the two groups"""
    
    #table with the two relevant columns
    reduced = table.select(label, group_label)  
    
    # table containing group means
    means_table = reduced.group(group_label, np.average)
    # array of group means
    means = means_table.column(1)
    
    return means.item(1) - means.item(0)

In [ ]:
difference_of_means(births, 'Birth Weight', 'Maternal Smoker')

# Random Permutation (Shuffling)

In [ ]:
letters = Table().with_column('Letter', make_array('a', 'b', 'c', 'd', 'e'))

In [ ]:
letters.sample()

In [ ]:
letters.sample(with_replacement = False)

In [ ]:
letters.with_column('Shuffled', letters.sample(with_replacement = False).column(0))

# Simulation Under Null Hypothesis

In [ ]:
smoking_and_birthweight

In [ ]:
shuffled_labels = smoking_and_birthweight.sample(with_replacement=False
                                                ).column('Maternal Smoker')

In [ ]:
original_and_shuffled = smoking_and_birthweight.with_column(
    'Shuffled Label', shuffled_labels
)

In [ ]:
original_and_shuffled

In [ ]:
difference_of_means(original_and_shuffled, 'Birth Weight', 'Shuffled Label')

In [ ]:
difference_of_means(original_and_shuffled, 'Birth Weight', 'Maternal Smoker')

# Permutation Test

## Write a function that does one single simulation of a test statistic

In [ ]:
def one_simulated_difference(table, label, group_label):
    """Takes: name of table, column label of numerical variable,
    column label of group-label variable
    Returns: Difference of means of the two groups after shuffling labels"""
    
    # array of shuffled labels
    shuffled_labels = table.sample(with_replacement = False
                                                    ).column(group_label)
    
    # table of numerical variable and shuffled labels
    shuffled_table = table.select(label).with_column(
        'Shuffled Label', shuffled_labels)
    
    return difference_of_means(shuffled_table, label, 'Shuffled Label')   

In [ ]:
one_simulated_difference(births, 'Birth Weight', 'Maternal Smoker')

## Run the simulation for 2500 tests

In [ ]:
differences = make_array()

for i in np.arange(2500):
    new_difference = one_simulated_difference(births, 'Birth Weight', 'Maternal Smoker')
    differences = np.append(differences, new_difference)

In [ ]:
Table().with_column('Difference Between Group Means', differences).hist()
print('Observed Difference:', observed_difference)
plots.title('Prediction Under the Null Hypothesis');